In [2]:
import pandas as pd
import os
import sys
import boto3
import io
import geopandas as gpd
import s3fs
import dask_geopandas
import dask.dataframe as dd
import matplotlib.pyplot as plt
import os

# suppress pandas purely educational warnings
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

sys.path.append(os.path.expanduser('../../'))
from scripts.utils.file_helpers import pull_gpkg_from_directory, upload_csv_aws
from scripts.utils.write_metadata import append_metadata

In [3]:
fs = s3fs.S3FileSystem()
bucket = 'ca-climate-index'
path = '2b_reproject/' 
pqt_list = [
    'governance/natural_resource_conservation/usda/forest_to_faucets/F2F2_Assessment/governance_usda_watershed_risk.parquet.gzip'
]

In [4]:
for pqt in pqt_list:
    ppath = path+pqt
    bucket_uri = f's3://{bucket}/{ppath}'
    print(pqt)
    df = gpd.read_parquet(bucket_uri)
    display(df)

governance/natural_resource_conservation/usda/forest_to_faucets/F2F2_Assessment/governance_usda_watershed_risk.parquet.gzip


,hilbert_distance,HUC_8,ACRES,HU_8_STATE,FIPS_C,latitude,longtitude,HUC8,Domestic,Industrial,...,Thermo_GW,Public_sup_GW,Aquacult_GW,Total_GW,Shape_Length,Shape_Area,geometry,index_right,USCB_GEOID,USCB_NAME
__null_dask_index__,,,,,,,,,,,,,,,,,,,,,
0,849944098,18070203,1084218.0,CA,06071 06037 06059 06065,34.030,-117.371,18070203,3.526,6.698,...,0.965,88.942,1.452,118.730,5.287693,0.428329,"MULTIPOLYGON (((-117.56759 34.38448, -117.5673...",3981599069,06059063500,635
1,849944098,18070203,1084218.0,CA,06071 06037 06059 06065,34.030,-117.371,18070203,3.526,6.698,...,0.965,88.942,1.452,118.730,5.287693,0.428329,"MULTIPOLYGON (((-117.56759 34.38448, -117.5673...",3981615326,06059063603,636.03
2,849944098,18070203,1084218.0,CA,06071 06037 06059 06065,34.030,-117.371,18070203,3.526,6.698,...,0.965,88.942,1.452,118.730,5.287693,0.428329,"MULTIPOLYGON (((-117.56759 34.38448, -117.5673...",3981618360,06059063604,636.04
3,849944098,18070203,1084218.0,CA,06071 06037 06059 06065,34.030,-117.371,18070203,3.526,6.698,...,0.965,88.942,1.452,118.730,5.287693,0.428329,"MULTIPOLYGON (((-117.56759 34.38448, -117.5673...",3981947055,06059099240,992.40
4,849944098,18070203,1084218.0,CA,06071 06037 06059 06065,34.030,-117.371,18070203,3.526,6.698,...,0.965,88.942,1.452,118.730,5.287693,0.428329,"MULTIPOLYGON (((-117.56759 34.38448, -117.5673...",3981946192,06059099239,992.39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10937,1331352993,18010204,1935539.0,CA OR,41035 41037 06093 06049,41.957,-121.308,18010204,0.769,0.190,...,0.000,2.531,0.147,98.573,6.491099,0.850621,"MULTIPOLYGON (((-121.77143 42.45282, -121.7714...",1694549954,06093001200,12
10938,1331352993,18010204,1935539.0,CA OR,41035 41037 06093 06049,41.957,-121.308,18010204,0.769,0.190,...,0.000,2.531,0.147,98.573,6.491099,0.850621,"MULTIPOLYGON (((-121.77143 42.45282, -121.7714...",1702239458,06093000100,1
10939,1331352993,18010204,1935539.0,CA OR,41035 41037 06093 06049,41.957,-121.308,18010204,0.769,0.190,...,0.000,2.531,0.147,98.573,6.491099,0.850621,"MULTIPOLYGON (((-121.77143 42.45282, -121.7714...",1521610571,06093000200,2


In [5]:
df.columns

Index(['hilbert_distance', 'HUC_8', 'ACRES', 'HU_8_STATE', 'FIPS_C',
       'latitude', 'longtitude', 'HUC8', 'Domestic', 'Industrial',
       'Irrigation', 'Livestock', 'Mining', 'Thermo', 'Public_sup',
       'Aquacultur', 'Total_SW', 'Ps_del_dom', 'Domestic_GW', 'Industri_GW',
       'Irrigati_GW', 'Livestoc_GW', 'Mining_GW', 'Thermo_GW', 'Public_sup_GW',
       'Aquacult_GW', 'Total_GW', 'Shape_Length', 'Shape_Area', 'geometry',
       'index_right', 'USCB_GEOID', 'USCB_NAME'],
      dtype='object')